In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import numpy as np
import os
import cv2
import tensorflow as tf
import tensorflow_datasets as tfds
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_dir = '/kaggle/input/flowers-recognition/flowers'
labels = ['daisy','dandelion','rose','sunflower','tulip']
x = []
y = []
for label in labels:
    data = os.path.join(data_dir,label)
    for image in os.listdir(data):
        try:
            im = cv2.imread(os.path.join(data,image),cv2.IMREAD_COLOR)
            im = cv2.resize(im,(128,128))
            # Using the Canny filter with different parameters
            
            x.append(im)
            y.append(labels.index(label))
            
            
        except Exception as e:
            pass


In [ ]:
np.unique(y)

In [ ]:
import matplotlib.pyplot as plt
import random
#fig, ax = plt.subplots(5, 2)
#fig.set_size_inches(12,15)
#for i in range(5):
 #   for j in range(2):
  #      ran =random.randint(0,len(x))
   #     ax[i, j].imshow(x[ran])
   #     ax[i, j].set_title(labels[y[ran]])

In [ ]:
x = np.array(x)/255.0
y = np.array(y)
x =x.reshape(-1,128,128,3)
y= y.reshape(-1,1)
y= tf.keras.utils.to_categorical(y,5,)

In [ ]:
datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
fill_mode= 'nearest',
horizontal_flip=False,
vertical_flip=False,
shear_range=0.1,
zoom_range = 0.1, # Randomly zoom image 
width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
height_shift_range=0.2
)
datagenerator.fit(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y)


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64,(5,5),padding ='same',strides=(2,2),activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128,(5,5),padding ='same',strides=(2,2),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(256,(5,5),padding ='same',strides=(2,2),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(512,(5,5),padding ='same',strides=(2,2),activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(5,activation='softmax')
])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=4,restore_best_weights=True)

In [ ]:
history =model.fit(datagenerator.flow(x_train,y_train,batch_size=32),epochs=20,callbacks=[earlystop],validation_data=datagenerator.flow(x_test,y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train','val'],loc='upper_right')



In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'],loc='upper_right')

In [ ]:
train_loss, train_acc = model.evaluate(x_train, y_train)
test_loss, test_acc = model.evaluate(x_test, y_test)

print("Accuracy on train set is %f" %(train_acc * 100)  + "%")
print("Accuracy on validation set is %f" %(test_acc * 100)  + "%")

In [ ]:
from tensorflow.keras.applications import VGG19
vgg = VGG19(include_top=False,weights = 'imagenet',classes=5,input_shape=(128,128,3))

for layer in vgg.layers[:17]:
    layer.trainable=False

from keras.regularizers import l2    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Conv2D(128,(5,5),padding ='same',strides=(2,2),activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2,2)))
model.add(tf.keras.layers.Conv2D(256,(5,5),padding ='same',strides=(2,2),activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(5,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history =model.fit(datagenerator.flow(x_train,y_train,batch_size=64),steps_per_epoch=len(x_train) / 32,epochs=20,callbacks=[earlystop],validation_data=datagenerator.flow(x_test,y_test))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train','val'],loc='upper_right')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Acc')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'],loc='upper_right')

In [ ]:
train_loss, train_acc = model.evaluate(x_train, y_train)
test_loss, test_acc = model.evaluate(x_test, y_test)

print("Accuracy on train set is %f" %(train_acc * 100)  + "%")
print("Accuracy on validation set is %f" %(test_acc * 100)  + "%")

In [ ]:
#As we can see pre-trained model is much accurate.